In [ ]:
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import finish_build
from ompl import geometric as og
from cloth_tools.ompl.single_arm_planner import SingleArmOmplPlanner

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(resources_root) / "robots" / "ur5e" / "ur5e.urdf"
robotiq_2f_85_gripper_urdf = Path(resources_root) / "grippers" / "2f_85_gripper" / "urdf" / "robotiq_2f_85_static.urdf"
table_urdf = "table.urdf"

arm_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
gripper_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper")
table_index = parser.AddModelFromFile(str(table_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_tool_frame = plant.GetFrameByName("tool0", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)
table_frame = plant.GetFrameByName("base_link", table_index)

X_W_B = RigidTransform(rpy=RollPitchYaw([0, 0, -np.pi / 2]), p=[0, 0, 0])

plant.WeldFrames(world_frame, arm_frame, X_W_B)
plant.WeldFrames(arm_tool_frame, gripper_frame, RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, 0, 0]))
plant.WeldFrames(world_frame, table_frame)

In [ ]:
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index],
    edge_step_size=0.125, # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding = 0.005,
    self_collision_padding = 0.005,
)

In [ ]:
ompl_planner = SingleArmOmplPlanner(is_state_valid_fn=collision_checker.CheckConfigCollisionFree)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
goal_joints = np.deg2rad([180, -135, 95, -50, -90, -90])

ompl_planner._set_start_and_goal_configurations(start_joints, goal_joints)

In [ ]:
simple_setup = ompl_planner._simple_setup
space_info = simple_setup.getSpaceInformation()

In [ ]:
from ompl import tools as ot

benchmark = ot.Benchmark(simple_setup, "twist_base_90")
benchmark.addPlanner(og.RRT(space_info))
benchmark.addPlanner(og.RRTstar(space_info))
benchmark.addPlanner(og.RRTConnect(space_info))
benchmark.addPlanner(og.TRRT(space_info))
benchmark.addPlanner(og.LazyRRT(space_info))
benchmark.addPlanner(og.KPIECE1(space_info))
benchmark.addPlanner(og.LBKPIECE1(space_info))

request = ot.Benchmark.Request()
request.runCount = 5
request.maxTime = 10.0

In [ ]:
benchmark.benchmark(request)

In [ ]:
print(benchmark.results())

In [ ]:
benchmark.saveResultsToFile()

Now use the `ompl_benchmark_statistics.py` script to generate a database file that we can upload to plannerarena.org:

```
python ompl_benchmark_statistics.py "ompl_idlab185_2024-01-23 11:12:48.log" benchmark.db
```